In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import all of our files
import sys
sys.path.append('../')

import fico
import distribution_to_loans_outcomes as dlo

DATA_DIR = '../data/'

Populating the interactive namespace from numpy and matplotlib


In [3]:
# set plotting parameters
sns.set_context("talk")
sns.set_style("white")

# this needs to be here so we can edit figures later
plt.rcParams['pdf.fonttype'] = 42

In [4]:
all_cdfs, performance, totals = fico.get_FICO_data(data_dir=DATA_DIR);

In [5]:
cdfs = all_cdfs[["White","Black"]]

# B is White
# A is Black

cdf_B = cdfs['White'].values
cdf_A = cdfs['Black'].values

repay_B = performance['White']
repay_A = performance['Black']

scores = cdfs.index
scores_list = scores.tolist()
scores_repay = cdfs.index

In [6]:
# to populate group distributions
def get_pmf(cdf):
    pis = np.zeros(cdf.size)
    pis[0] = cdf[0]
    for score in range(cdf.size-1):
        pis[score+1] = cdf[score+1] - cdf[score]
    return pis

# to get loan repay probabilities for a given score
loan_repaid_probs = [lambda i: repay_A[scores[scores.get_loc(i,method='nearest')]], 
                     lambda i: repay_B[scores[scores.get_loc(i,method='nearest')]]]

In [7]:
# basic parameters
N_scores = cdf_B.size
N_groups = 2

# get probability mass functions of each group
pi_A = get_pmf(cdf_A)
pi_B = get_pmf(cdf_B)
pis = np.vstack([pi_A, pi_B])

# demographic statistics 
group_ratio = np.array((totals["Black"], totals["White"]))
group_size_ratio = group_ratio/group_ratio.sum()
print(group_size_ratio)

[0.12066905 0.87933095]


In [8]:
# to get loan repay probabilities for a given score
loan_repaid_probs = [lambda i: repay_A[scores[scores.get_loc(i,method='nearest')]], 
                     lambda i: repay_B[scores[scores.get_loc(i,method='nearest')]]]

# unpacking repay probability as a function of score
loan_repay_fns = [lambda x: loan_repaid_prob(x) for
                      loan_repaid_prob in loan_repaid_probs]

In [9]:
# all of the above is from Lydia's code in delayed-impact repos
# all of the below is my code transforming the data and running linear regression on it!
# NOTE: references for certain chunks will be included

In [10]:
import pandas as pd

In [11]:
def get_repay_probabilities(samples, repay_probs):
    sample_probs = []
    for index, score in enumerate(samples):
        prob_index = np.where(scores_arr == score)
        repay_prob = repay_probs[prob_index[0][0]]
        sample_probs.insert(index, repay_prob)
    return sample_probs

In [12]:
# Note: much of the linear regression code is adapted from the below link
#       https://realpython.com/linear-regression-in-python/
# Problem with the below: https://datascience.stackexchange.com/questions/18258/how-to-force-weights-to-be-non-negative-in-linear-regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

def run_linear_regression(x_arr, y_arr):
    # data prep
    x = x_arr.reshape(-1,1)
    y = y_arr
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
    
    # train model
    model = LinearRegression().fit(x_train, y_train)

    # analyze regression model on training set
    # NOTE: the intercept, b0 value or coefficient, tells us what the regression predicts if the x is zero
    #       the slope, b1 value, tells us how the predicted response rises/increases when x is increased by 1
    # QUESTION: not sure if the input for score here should be our train and test x and y values? or our test values?
    r_sq = model.score(x_train, y_train)
    print('coefficient of determination:', r_sq)
    print('intercept:', model.intercept_)
    print('slope:', model.coef_)
    
    # get predictions
    y_pred = model.predict(x_test)
    print('predicted response:', y_pred, sep='\n')
    
    return model, y_test, y_pred

In [13]:
# NOTE: this function came from the Naomi Fridman's answer here 
#       https://stackoverflow.com/questions/26319259/how-to-get-a-regression-summary-in-python-scikit-like-r-does

from sklearn import metrics

def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [14]:
# Convert data in format needed
# Make repay probabilities into percentages from decimals
# NOTE: A is Black, B is White

scores_arr = np.asarray(scores_list)
repay_A_arr = pd.Series.to_numpy(repay_A)*100
repay_B_arr = pd.Series.to_numpy(repay_B)*100

In [15]:
# Sample data according to the pmf
# Reference: https://www.w3schools.com/python/ref_random_choices.asp

from random import choices

num_A_samples = 120
num_B_samples = 880

samples_A = np.asarray(sorted(choices(scores_arr, pi_A, k=num_A_samples)))
samples_B = np.asarray(sorted(choices(scores_arr, pi_B, k=num_B_samples)))

In [16]:
# Calculate samples groups' probabilities and make arrays for race
# A == Black == 0 (later defined as 0.0 when converting to pandas df)
# B == White == 1 (later defined as 1.0 when converting to pandas df)

samples_A_probs = np.asarray(get_repay_probabilities(samples=samples_A, repay_probs=repay_A_arr))
samples_A_race = np.asarray([0] * num_A_samples)

samples_B_probs = np.asarray(get_repay_probabilities(samples=samples_B, repay_probs=repay_B_arr))
samples_B_race = np.asarray([1] * num_B_samples)

# If needed, I can use pandas df like below
'''
data_A_dict = {'score': samples_A, 'repay_probability': samples_A_probs, 'race': samples_A_race}
data_B_dict = {'score': samples_B, 'repay_probability': samples_B_probs, 'race': samples_B_race}

data_A_df = pd.DataFrame(data=data_A_dict, dtype=np.float64)
data_B_df = pd.DataFrame(data=data_B_dict, dtype=np.float64)

data_all_df = pd.concat([data_A_df, data_B_df], ignore_index=True)
print(data_all_df)
data_all_df_shuffled = data_all_df.sample(frac=1).reset_index(drop=True)
print(data_all_df_shuffled)
'''

"\ndata_A_dict = {'score': samples_A, 'repay_probability': samples_A_probs, 'race': samples_A_race}\ndata_B_dict = {'score': samples_B, 'repay_probability': samples_B_probs, 'race': samples_B_race}\n\ndata_A_df = pd.DataFrame(data=data_A_dict, dtype=np.float64)\ndata_B_df = pd.DataFrame(data=data_B_dict, dtype=np.float64)\n\ndata_all_df = pd.concat([data_A_df, data_B_df], ignore_index=True)\nprint(data_all_df)\ndata_all_df_shuffled = data_all_df.sample(frac=1).reset_index(drop=True)\nprint(data_all_df_shuffled)\n"

In [17]:
# TROUBLESHOOTING: check range of repay probability values in array

max_val = np.max(samples_A_probs)
min_val = np.min(samples_A_probs)
print('the range of the Group A (Black) repay probabilities is: ', max_val-min_val)
print('the min value is: ', min_val)
print('the max value is: ', max_val)

the range of the Group A (Black) repay probabilities is:  94.86
the min value is:  0.769999999999996
the max value is:  95.63


In [18]:
# Run linear regression on Group A -- Black

model_A, y_test_A, y_pred_A = run_linear_regression(samples_A, samples_A_probs)

coefficient of determination: 0.8981214374536469
intercept: -96.74663213655134
slope: [0.25379432]
predicted response:
[ 21.95102126  23.90328529 -14.56561271  33.04185796   3.86468944
  35.45129775  41.07332393  -8.5228907   19.0226252   78.15076118
  -2.17803258  33.84500456  -6.71007409  13.46590331  56.37594376
  58.49089647  79.53007816 -17.58697372  55.26559359  72.63349325
  35.45129775  22.92715328   6.41606096  39.46703074  19.99875722
  -5.19939359  -3.68871308  90.6132958   45.35175278  63.24954005
   3.86468944  46.0126755    2.35400893  61.66332552  33.04185796
  80.44962282]


In [19]:
# Analysis on Group A linear regression predictions
# NOTE: can't get regression results because some predicted values are negative...

regression_results(y_test_A, y_pred_A)

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [20]:
# Run linear regression on Group B -- White

model_B, y_test_B, y_pred_B = run_linear_regression(samples_B, samples_B_probs)

coefficient of determination: 0.9003970054749643
intercept: -77.68698171233552
slope: [0.23480279]
predicted response:
[ 70.82578172  59.8927769  108.44624108  68.37991934  55.00105215
 101.0157731   94.96212713  84.54813306 109.93233467  12.60028083
  81.57056147 101.7588199  118.80587774  70.82578172  96.68861822
   7.00973825  88.74675921  52.55518977  33.93465137 103.98796029
 100.6442497   81.1451941  108.44624108  23.19125314  15.39555211
 101.3872965  109.56081127  26.45240298 101.0157731   23.19125314
   9.80500954 109.18928787  29.41921313  81.57056147  97.37921466
  64.46653954  66.42322944  24.27830309 109.56081127  90.4732503
  90.81854852  37.54700195  67.89074687  47.59070836  40.90328718
  40.90328718  26.45240298  93.92623248  66.42322944  89.78265387
  90.81854852  89.09205743 110.56718343  96.68861822 117.77604095
  88.401461    82.84666358  95.30742535  98.06981109  69.35826429
 109.56081127  98.06981109  67.40157439 110.56718343  99.15815611
  30.32230078  93.926232

In [21]:
# Analysis on Group B linear regression predictions
# NOTE: can't get regression results because some predicted values are negative...

regression_results(y_test_B, y_pred_B)

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [ ]:
# TODO: try the NNLS regression from scikit learn
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_nnls.html

In [ ]:
# try scipy non-negative least square regression
# Reference: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.nnls.html
# TODO: look more into this if the above doesn't pan out...

#from scipy.optimize import nnls

#solution_vector, residual = nnls(x_train_B, x_test_B)
#print(solution_vector)
#print(residual)

In [ ]:
# potential TODO: could also try the below
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.lsq_linear.html